In [ ]:
import requests
from lxml import etree
import re

class Adjunto:
    name: str
    index: str

    def __init__(self, name: str, index: str) -> None:
        self.name = name
        self.index = index    

    def download(self, base_url: str) -> requests.Response:
        url = f"{base_url}/RFB/DownloadFile.aspx?index={self.index}"
        response = requests.get(url)
        if response.status_code != 200:
            raise ValueError(f"Error al descargar el adjunto: {self.name}")
        return response

class Licitacion:
    parser = etree.HTMLParser()
    ficha: etree.ElementBase
    adjuntos: etree.ElementBase
    url: str

    base_url = "https://www.mercadopublico.cl/Procurement/Modules"

    def __init__(self, id_licitacion: str) -> None:
        self.url = (
            self.base_url
            + f"/RFB/DetailsAcquisition.aspx?idlicitacion={id_licitacion}"
        )
        response: requests.Response = requests.get(self.url)
        if response.status_code != 200:
            raise ValueError(f"Error al acceder a la URL: {self.url}")
        self.ficha: etree.ElementBase = etree.fromstring(
            response.text, self.parser
        )

    def get_estado(self) -> str:
        imgEstado = self.ficha.find(".//img[@id='imgEstado']", None)
        estado = (
            match.group(1)
            if (match := re.match(r".*/([^/]+).png$", imgEstado.get("src", "")))
            else "desconocido"
        )
        return estado

    def get_adjuntos(self) -> list[tuple[str, str]]:
        return []

In [9]:
id_licitacion = "4127-83-LR24"
ficha = Licitacion(id_licitacion)
estado = ficha.get_estado()
print(f"Estado de la licitación {id_licitacion}: {estado}")

Estado de la licitación 4127-83-LR24: desierta


## Estado licitación

## Adjuntos

In [ ]:
def get_url_adjuntos(tree: Licitacion) -> str:
    box_b: Licitacion = tree.find(".//body//div[@id='box_b']", None)
    if box_b is None:
        return ""
    imgAdjuntos: Licitacion = box_b.find(".//input[@id='imgAdjuntos']", None)
    if imgAdjuntos is None:
        return ""
    enc_adjuntos = ( 
        match.group(1)
        if (
            match := re.match(
                r".*enc=([^']+)'.*", imgAdjuntos.get("onclick", "")
            )
        )
        else ""
    ) # varía cada vez que se obtiene la página de la licitación
    if enc_adjuntos == "":
        return ""
    url_adjuntos = f"https://www.mercadopublico.cl/Procurement/Modules/Attachment/ViewAttachment.aspx?enc={enc_adjuntos}"
    return url_adjuntos

url_adjuntos = get_url_adjuntos(tree)
print(url_adjuntos)  # URL de los adjuntos

https://www.mercadopublico.cl/Procurement/Modules/Attachment/ViewAttachment.aspx?enc=8NEANHfNuX74LECZ3i%2fwojJIs%2bK33UZTSCQ086ko8go00TyvSq4puIPuf%2bm0t%2bPRIelMf8jx6Ylxk%2fzBT%2bn8ZA2vldAjiS3cgkZq2tZXI0Fqb9OH%2fcBoFkZ4OOxlhUlvH5tH3Bb%2bdCLOpUbVaC4E8bk%2fU4BHxGd5KXN4sHlyWCMRABw3C7WNL236t8tdid1kXGcSR9rLmSI2s35KFM0hXUlTjVhfWt6mtG4HZyYu8hyzg0HKeuvALsSBA0fQ1VRDlVK51E%2fk0zdbjiE%2b9jW2LcUVrMiLL3onF62GeMoiqlcT0pHhBNzMHK4GAsqedUqH


In [126]:
session = requests.Session()

response_adj = session.get(url_adjuntos)
tree_adj = etree.fromstring(response_adj.text, parser)
print(etree.tostring(tree_adj))

b'<html xmlns="http://www.w3.org/1999/xhtml" xmlns="http://www.w3.org/1999/xhtml">&#13;\n<head id="Head1"><meta http-equiv="Content-Type" content="text/html; charset=ASCII" /><title>&#13;\n\tVer anexos&#13;\n</title><link id="lnkStyleSheet" href="../../Includes/styles/ChileCompra.css" type="text/css" rel="stylesheet" /> &#13;\n\t<script src="../../Includes/scripts/chilecompra.js" type="text/javascript"></script>   &#13;\n    <script src="../../Includes/scripts/JQuery/jquery-1.7.1.js" type="text/javascript"></script>&#13;\n\t<!-- Global Site Tag (gtag.js) - Google Analytics -->&#13;\n    <script async="" src="https://www.googletagmanager.com/gtag/js?id=UA-51338325-2"></script>&#13;\n    <script src="../../Includes/scripts/analytics.js" type="text/javascript"></script>&#13;\n    <script src="../../Includes/scripts/hotjar.js" type="text/javascript"></script>    &#13;\n</head>&#13;\n<body>&#13;\n   <form name="form1" method="post" action="./ViewAttachment.aspx?enc=8NEANHfNuX74LECZ3i%2fwojJ

In [ ]:
def get_next_adj_page(tree_adj: Licitacion, page: int, url_adjuntos) -> list:
  view_state = tree_adj.find(".//input[@id='__VIEWSTATE']", None).get("value")
  view_state_generator = tree_adj.find(".//input[@id='__VIEWSTATEGENERATOR']", None).get("value")

  headers = {'User-Agent': 'Mozilla/5.0'}
  payload = {
  "__VIEWSTATE": view_state,
  "__VIEWSTATEGENERATOR": view_state_generator,
  "__EVENTARGUMENT": f"Page${page}",
  }
  response = session.post(url_adjuntos, data=payload, headers=headers)
  if response.status_code != 200:
    return []
  tree_adj = etree.fromstring(response.text, parser)
  tabla_adjuntos: Licitacion = tree_adj.find(".//body//table[@id='DWNL_grdId']", None)
  if tabla_adjuntos is None:
    return []
  filas_adjuntos = tabla_adjuntos.getchildren()
  return filas_adjuntos

def listar_adjuntos(tree_adj, url_adjuntos):
  tabla_adjuntos: Licitacion = tree_adj.find(".//body//table[@id='DWNL_grdId']")

  filas_adjuntos = tabla_adjuntos.getchildren()
  adjuntos = []
  i = 1
  page = 1
  while i < len(filas_adjuntos):
    adjunto_td = filas_adjuntos[i][6] # Columna 6 corresponde a "Acciones" (la lupa para descargar archivo)
    adjunto_name = filas_adjuntos[i][1].find(".//span").text.strip() # Columna 1 corresponde a "Anexo"
    # Replace all spaces with _
    adjunto_name = re.sub(r'\s+', '_', adjunto_name)

    adjunto_search = adjunto_td.find(".//input[@type='image']").get("name")
    adjuntos.append({"adjunto_name": adjunto_name, "adjunto_search": adjunto_search})
    i += 1
    if i > 100:
      page += 1
      filas_adjuntos = get_next_adj_page(tree_adj, page, url_adjuntos)
      i = 1

  return adjuntos

In [166]:
print(listar_adjuntos(tree_adj, url_adjuntos))  # Lista de adjuntos con nombre y búsqueda

[{'adjunto_name': '1730142935191_ACTA_EVALUACION_EQUIPAMIENTO.pdf', 'adjunto_search': 'DWNL$grdId$ctl02$search'}, {'adjunto_name': '1731001882474_RES._EXENTA_N_834_DECLARA_DESIERTA_LICITACION.pdf', 'adjunto_search': 'DWNL$grdId$ctl03$search'}, {'adjunto_name': '1725303188073_MARCELA_MIRANDA.pdf', 'adjunto_search': 'DWNL$grdId$ctl04$search'}, {'adjunto_name': '1725303116253_ERIK_MORENO.pdf', 'adjunto_search': 'DWNL$grdId$ctl05$search'}, {'adjunto_name': '1725303207025_ALEJANDRO_GRIMAU.pdf', 'adjunto_search': 'DWNL$grdId$ctl06$search'}, {'adjunto_name': '1725303165877_IVAN_ROBLES.pdf', 'adjunto_search': 'DWNL$grdId$ctl07$search'}, {'adjunto_name': 'ACLARATORIA_4127-83-LR24.pdf', 'adjunto_search': 'DWNL$grdId$ctl08$search'}, {'adjunto_name': 'RES.EX.17131_DESGINA_COMISIÓN_4127-83-LR24.pdf', 'adjunto_search': 'DWNL$grdId$ctl09$search'}, {'adjunto_name': 'ANEXOS_EN_WORD_COMPUTADORES_37.000.000.000.docx', 'adjunto_search': 'DWNL$grdId$ctl10$search'}, {'adjunto_name': 'OF._E527503_RES._AF._N_

In [173]:
test_adj = etree.fromstring(session.get("https://www.mercadopublico.cl/Procurement/Modules/Attachment/ViewAttachment.aspx?enc=pwNgzYFXxJElmU5u02aBG7IhhQ91MntV%2bNd2bvuEpclt9qj9qcErDawryYrPnwNG3ztF2Varz3FusF7nBlcy6S%2f5GVDJFYCy6Mh7Iu6uDjGfFZK8EdoXzaZJkiuHDhUkIhsrkK0XPVnq4wnIAYX%2biG%2fDlyYYWLjvsVGBoKpFAIkDZuy7tUGJ8bQbBbqX3rjExaQXkz9jqtXuPZOodAY%2fLtnRjeQ%2fBH%2bOW74Ewo2SMZIDlNbEb6ogLCVTrimptNK6TkKdBvyOtzK9OUAG9o6ZQ29QZT22CqpW%2bYbHb5TIDy6CWRWkV7zm16VFdDtiPWyF").text, parser)
test_list = listar_adjuntos(test_adj, url_adjuntos) # Prueba de paginación
print(len(test_list))  # Debería ser mayor a 100 si hay más de una página

112


In [129]:
def descargar_adjuntos(tree_adj, lista_adj, session, url_adjuntos):
  view_state = tree_adj.find(".//input[@id='__VIEWSTATE']").get("value")
  view_state_generator = tree_adj.find(".//input[@id='__VIEWSTATEGENERATOR']").get("value")

  headers = {'User-Agent': 'Mozilla/5.0'}
  base_payload = {
  "__VIEWSTATE": view_state,
  "__VIEWSTATEGENERATOR": view_state_generator,
  }

  for adj in lista_adj:
    payload = base_payload.copy()
    payload[adj["adjunto_search"]+".x"] = "0"
    payload[adj["adjunto_search"]+".y"] = "0"
    response = session.post(url_adjuntos,headers=headers,data=payload)

    print(adj["adjunto_name"], response.status_code)
    if response.status_code != 200:
      continue
    with open(adj["adjunto_name"], "w+b") as f:
      # print response content length
      print("\tsize:", len(response.content))
      f.write(response.content)


In [130]:
# adjuntos = listar_adjuntos(filas_adjuntos)
# descargar_adjuntos(tree_adj, adjuntos, session)

## Listar licitaciones

In [ ]:
listar_session = requests.Session()
listar_url = "https://www.mercadopublico.cl/BuscarLicitacion/Home/Buscar"
listar_payload = {
    "textoBusqueda": "",
    "idEstado": "-1",
    "codigoRegion": "-1",
    "idTipoLicitacion": "25",
    "fechaInicio": "1990-01-01T03:00:00.000Z",
    "fechaFin": "2025-05-26T04:00:00.000Z",
    "registrosPorPagina": "10",
    "idTipoFecha": [],
    "idOrden": "3",
    "compradores": [],
    "garantias": None,
    "rubros": [],
    "proveedores": [],
    "montoEstimadoTipo": [0],
    "esPublicoMontoEstimado": None,
    "pagina": 1,
}
listar_response = listar_session.post(listar_url, data=listar_payload)
print(listar_response.text)
listar_tree: Licitacion = etree.fromstring(listar_response.text, parser)


<input type="hidden" id="hdnTotalPresupuestoPublico" name="hdnTotalPresupuestoPublico" value="22500" />
<input type="hidden" id="hdnTotalPresupuestoPrivado" name="hdnTotalPresupuestoPrivado" value="24718" />
    <div>
            <div class="cabecera-resultados margin-bottom-lg row">
                <div class='col-md-6'>
                    <div class="margin-bottom-xs">
                        <span class='rot-resultado'>Se han encontrando más de</span><strong><span class='n-result'>47.218</span>resultados </strong><span class='rot-resultado'>para la búsqueda.</span>
                    </div>
                </div>
                <div class="col-md-6">
                    <div class="select-wrap esconder-cont">
                        <div class="contenedor-ordenar">
                            <label for="ordenarpor">Ordenar por </label>
                            <select id="ordenarpor" name="ordenarpor" class="form-control"></select>
                        </div>
            

In [ ]:
# n_resultados = int(listar_tree.find(".//span[@class='n-result']", None).text.strip().replace(".", ""))
resultados = []
for n_pag in range(1, 1002):
    listar_payload["pagina"] = n_pag
    listar_response = listar_session.post(listar_url, data=listar_payload)
    listar_tree: Licitacion = etree.fromstring(listar_response.text, parser)
    resultados_pag = listar_tree.findall(".//div[@class='responsive-resultado']", None)
    if resultados_pag is None or len(resultados_pag) == 0:
        print("No se encontraron resultados en la página", n_pag)
        break
    for res in resultados_pag:
        id = res.find(".//span[@class='clearfix']").text.strip()
        # url = res.find(".//div[@class='lic-block-body']//a").get("onclick")
        url = "http://www.mercadopublico.cl/Procurement/Modules/RFB/DetailsAcquisition.aspx?idlicitacion=" + id
        # url = (
        #     match.group(1)
        #     if (match := re.match(r".*\$.Busqueda.verFicha\('([^']+)'\).*", url))
        #     else url
        # )
        resultados.append((id, url))
        print(id, url)
    n_pag += 1

5005-1-LR25 http://www.mercadopublico.cl/Procurement/Modules/RFB/DetailsAcquisition.aspx?idlicitacion=5005-1-LR25
2582-52-LR25 http://www.mercadopublico.cl/Procurement/Modules/RFB/DetailsAcquisition.aspx?idlicitacion=2582-52-LR25
1019-65-LR25 http://www.mercadopublico.cl/Procurement/Modules/RFB/DetailsAcquisition.aspx?idlicitacion=1019-65-LR25
2793-88-LR25 http://www.mercadopublico.cl/Procurement/Modules/RFB/DetailsAcquisition.aspx?idlicitacion=2793-88-LR25
1057545-42-LR25 http://www.mercadopublico.cl/Procurement/Modules/RFB/DetailsAcquisition.aspx?idlicitacion=1057545-42-LR25
975-31-LR25 http://www.mercadopublico.cl/Procurement/Modules/RFB/DetailsAcquisition.aspx?idlicitacion=975-31-LR25
621-582-LR25 http://www.mercadopublico.cl/Procurement/Modules/RFB/DetailsAcquisition.aspx?idlicitacion=621-582-LR25
621-586-LR25 http://www.mercadopublico.cl/Procurement/Modules/RFB/DetailsAcquisition.aspx?idlicitacion=621-586-LR25
608-84-LR25 http://www.mercadopublico.cl/Procurement/Modules/RFB/Detai

In [145]:
for id, url in resultados[1000:1005]:
    print(f"ID: {id}, URL: {url}")
    lic_tree = etree.fromstring(listar_session.get(url).text, parser)
    adj_url = get_url_adjuntos(lic_tree)
    if not adj_url:
        print("No se encontraron adjuntos para la licitación", url)
        continue
    print("Adjuntos URL:", adj_url)
    adj_response = listar_session.get(adj_url)
    adj_tree = etree.fromstring(adj_response.text, parser)
    adjuntos = listar_adjuntos(adj_tree)
    print("Adjuntos encontrados:", adjuntos)

ID: 2981-49-LR25, URL: http://www.mercadopublico.cl/Procurement/Modules/RFB/DetailsAcquisition.aspx?idlicitacion=2981-49-LR25


Adjuntos URL: https://www.mercadopublico.cl/Procurement/Modules/Attachment/ViewAttachment.aspx?enc=Cc3uyeZlaaYLGx3u1oXaLwAOBWU%2fDGTd8J%2bA7ouYdcq4dSqWHJrGZ1vfNvv7Gz0xg6EZcvOfv2cvUJAKtI27ftJzg3LuxICeNvufU3grZQ%2f%2fVgMHqdnAN6%2b2Hyi%2fmw1Sxt%2bdLCn%2b%2blMgObmGObhoYhFcIKu4bNVms3C0szb9%2fuMOXk5eIvXrmcq3%2bpS%2fiexYyIUAz0KCXBGZBpFudCoOku8jPJYJS9WGvhOe3oxOx%2fA%2bAF%2bqu4hFjrSCN68I6%2bhs1V15l6bO4oSdvpgGwM27Ni3u1Q9lYhpbmGIfWSBXfYFXguMlTFCRygkRcgOdSmlU
Adjuntos encontrados: [{'adjunto_name': 'INFORME_JURIDICO.pdf', 'adjunto_search': 'DWNL$grdId$ctl02$search'}, {'adjunto_name': 'ACTA_DE_REUNION_Y_PROPOSICION_A_CAE.pdf', 'adjunto_search': 'DWNL$grdId$ctl03$search'}, {'adjunto_name': 'INFORME_DE_EVALUACION_VEHÍCULOS_FINAL.pdf', 'adjunto_search': 'DWNL$grdId$ctl04$search'}, {'adjunto_name': 'RESOLEX_328_DE_12.MAY.025.pdf', 'adjunto_search': 'DWNL$grdId$ctl05$search'}, {'adjunto_name': 'Acta_declaración_ausencia_de_conflicto_de_intereses_y_confidencia.pdf', 'adjunto_search': 'DWNL$grdId$ctl06$s

IndexError: list index out of range

In [ ]:
_, url = resultados[4]
lic_tree = etree.fromstring(listar_session.get(url).text, parser)
def get_url_apertura(lic_tree: Licitacion) -> str:
    imgAperturaElectronica:Licitacion = lic_tree.find(".//input[@id='imgAperturaElectronica']", None)
    if imgAperturaElectronica is None:
        return ""
    url_apertura = imgAperturaElectronica.get("href", "")
    if not url_apertura:
        return ""
    url_apertura = "https://www.mercadopublico.cl" + url_apertura
    return url_apertura

print(get_url_apertura(lic_tree))